In [55]:
# Install the datasets library from huggingface
!pip install opperai -U
!pip install pydantic


In [119]:
import os
from opperai import fn, Client, AsyncClient
from opperai.types.indexes import RetrievalResponse
print(os.environ.get("OPPER_API_KEY"))
from pydantic import BaseModel, Field
from typing import List, Literal
os.environ["OPPER_API_KEY"] = "op-AJ28BGSXFM887D1K7ZE7"



op-AJ28BGSXFM887D1K7ZE7


In [101]:
opper = Client()

# We get or create our index
index = opper.indexes.get(name="mistral-rag2")
if not index:
    index = opper.indexes.create(name="mistral-rag2")

print(index)

# we upload our pdf to the index
opper.indexes.upload_file(
    id=index.id, 
    #file_path="./NIPS-2017-attention-is-all-you-need-Paper.pdf"
    file_path="./reddit-sec.pdf"
    )


id=1652 name='mistral-rag2' created_at=datetime.datetime(2024, 5, 4, 21, 17, 47, 3506, tzinfo=TzInfo(UTC))


{'uuid': '6672572a-af9d-47d9-adeb-bd02da8e5a93',
 'key': '1/1652/6672572a-af9d-47d9-adeb-bd02da8e5a93',
 'original_filename': 'reddit-sec.pdf',
 'document_id': 14542}

In [114]:

question = "What are the key financial metrics of Reddit?"

# We retrieve the content from the index
results = opper.indexes.retrieve(
    id=index.id,
    query=question,
    k=3
)

print(results)


[RetrievalResponse(content='evolving needs of our Redditors and advertisers.\nFor a discussion of the key opportunities and challenges we face in growing our business, see “—Key Factors Affecting our Performance.”\nWe are headquartered in San Francisco, California, and have several offices around the world.\nKey Financial and Operating Metrics\nWe review a number of metrics, including the key metrics discussed below, to evaluate our business, measure our performance, identify trends\naffecting our business, formulate business plans, and make strategic decisions.\n96 04/05/2024, 23:15 Document\nhttps://www.sec.gov/Archives/edgar/data/1713445/000162828024006294/reddits-1q423.htm 106/281Table of Contents\nTrends in User Metrics\nDaily Active Unique. We define a daily active unique (“DAUq”) as a user whom we can identify with a unique identifier who has visited a page on\nthe Reddit website, www.reddit.com, or opened a Reddit application at least once during a 24-hour period. We calculate 

In [126]:

# We define our structured object for references
class Source(BaseModel):
    thought: str 
    file_name: str = Field(..., description="The name of the file")
    page_number: int = Field(..., description="The page number of the source")
    relevant_facts: List[str] = Field(..., description="The relevant facts from the source")
    relevance_score: Literal["High", "Medium", "Low"] 

@fn(path="test/mistral-rag", model="mistral/mistral-small-eu")
def get_source(query: str, content: RetrievalResponse) -> Source:
    """ Analyse the content with the query in mind and return a structured source"""

sources =[]   
for result in results:
    sources.append(get_source(question, result))
    
print(sources)



In [125]:
for source in sources:
    print("-")
    print(source.thought)
    print(source.file_name)
    print(source.page_number)
    print(source.relevant_facts)
    print(source.relevance_score)
    
    
    



-
The key financial metrics of Reddit include revenue, average revenue per unique (ARPU), daily active unique (DAUq), and weekly active unique (WAUq). The content mentions a revenue of $804.0 million for the year ended December 31, 2023, and an ARPU is not explicitly provided but can be calculated from the revenue and DAUq. The DAUq and WAUq are discussed in detail, with the DAUq growing 27% globally in the three months ended December 31, 2023, and the WAUq growing 29% globally in the same period.
reddit-sec.pdf
106
['96 04/05/2024, 23:15 Document', 'https://www.sec.gov/Archives/edgar/data/1713445/000162828024006294/reddits-1q423.htm 106/281Table of Contents', '97 04/05/2024, 23:15 Document', 'https://www.sec.gov/Archives/edgar/data/1713445/000162828024006294/reddits-1q423.htm 107/281Table of Contents', '99 04/05/2024, 23:15 Document', 'https://www.sec.gov/Archives/edgar/data/1713445/000162828024006294/reddits-1q423.htm 109/281Table of Contents']
High
-
The content doesn't contain spec